# Imports

In [1]:
import numpy as np
import pandas as pd
import matplotlib
from catboost import CatBoostRegressor
from sklearn.model_selection import cross_val_score
from sklearn.metrics import mean_absolute_error
import optuna

rng = np.random.RandomState(42)

# Data

In [2]:
data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
data = pd.read_csv(data_path)
data['DATE'] = data['DATE'].astype('datetime64[ns]')

X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
X_test = data.loc[data['DATE'].dt.year == 2023].drop(columns=['TARGET', 'DATE']).copy()
y_test = data.loc[data['DATE'].dt.year == 2023, 'TARGET'].copy()

print(f'Train: {X_train.shape}, {y_train.shape}')
print(f'Valid: {X_valid.shape}, {y_valid.shape}')
print(f'Test: {X_test.shape}, {y_test.shape}')

Train: (34938, 52), (34938,)
Valid: (2920, 52), (2920,)
Test: (2920, 52), (2920,)


# Baseline

In [3]:
model = CatBoostRegressor(verbose=100, random_state=42)
model.fit(X_train, y_train)

print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

Learning rate set to 0.071783
0:	learn: 10.1222909	total: 61.4ms	remaining: 1m 1s
100:	learn: 3.9090917	total: 608ms	remaining: 5.41s
200:	learn: 3.8082690	total: 1.11s	remaining: 4.41s
300:	learn: 3.7281364	total: 1.64s	remaining: 3.8s
400:	learn: 3.6601188	total: 2.18s	remaining: 3.25s
500:	learn: 3.6011532	total: 2.7s	remaining: 2.69s
600:	learn: 3.5470066	total: 3.17s	remaining: 2.1s
700:	learn: 3.5004606	total: 3.69s	remaining: 1.57s
800:	learn: 3.4523707	total: 4.21s	remaining: 1.04s
900:	learn: 3.4076202	total: 4.74s	remaining: 521ms
999:	learn: 3.3676583	total: 5.27s	remaining: 0us
Train MAE: 2.5486400384277177
Valid MAE: 3.1525589942969856


# Hyperparameter tuning

In [4]:
def load_data():
    data_path = 'https://raw.githubusercontent.com/antbartash/max_temp/master/data/data_features.csv'
    data = pd.read_csv(data_path)
    data['DATE'] = data['DATE'].astype('datetime64[ns]')
    X_train = data.loc[data['DATE'].dt.year <= 2021].drop(columns=['TARGET', 'DATE']).copy()
    y_train = data.loc[data['DATE'].dt.year <= 2021, 'TARGET'].copy()
    X_valid = data.loc[data['DATE'].dt.year == 2022].drop(columns=['TARGET', 'DATE']).copy()
    y_valid = data.loc[data['DATE'].dt.year == 2022, 'TARGET'].copy()
    return X_train, X_valid, y_train, y_valid

def objective(trial):
    PARAMS = {
        'n_estimators': trial.suggest_int('n_estimators', 50, 1000),
        'learning_rate': trial.suggest_float('learning_rate', 1e-6, 0.5),
        'depth': trial.suggest_int('depth', 1, 12),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 0.0, 200.0),
        'random_strength': trial.suggest_float('random_strength', 0.0, 200.0), # CPU only
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 200.0),
        'grow_policy': trial.suggest_categorical('grow_policy', ['SymmetricTree', 'Depthwise']),
    }
    X_train, _, y_train, _ = load_data()
    model = CatBoostRegressor(verbose=0, random_state=42)
    model.set_params(**PARAMS)
    score = np.mean(cross_val_score(model, X_train, y_train, cv=3, scoring='neg_mean_absolute_error'))
    return score

In [5]:
sampler = optuna.samplers.TPESampler(
    n_startup_trials = 50, # the random sampling is used instead of the TPE algorithm until the given number of trials finish in the same study
    n_ei_candidates = 24, # number of candidate samples used to calculate the expected improvement.
    multivariate = True, # multivariate TPE when suggesting candidates; default: False
    seed = 42
)
study = optuna.create_study(
    pruner=optuna.pruners.MedianPruner(n_startup_trials=10, n_warmup_steps=100, interval_steps=20),
    direction='maximize',
    sampler=sampler,
    storage='sqlite:///db.sqlite3'
)
study.optimize(
    objective, n_trials = 1200,
    timeout = 3600*10, # in seconds
    n_jobs = 1,
    show_progress_bar = True
)

/usr/local/lib/python3.10/dist-packages/optuna/_experimental.py:31: ExperimentalWarning: Argument ``multivariate`` is an experimental feature. The interface can change in the future.
  warnings.warn(
[I 2024-12-20 22:26:03,969] A new study created in RDB with name: no-name-2cf741bc-a57a-4f8a-8d0a-65d6aee3a2bc


  0%|          | 0/1200 [00:00<?, ?it/s]

[I 2024-12-20 22:26:47,154] Trial 0 finished with value: -3.2232916402730614 and parameters: {'n_estimators': 406, 'learning_rate': 0.47535720249065166, 'depth': 9, 'l2_leaf_reg': 119.73169683940732, 'random_strength': 31.203728088487303, 'bagging_temperature': 31.19890406724053, 'grow_policy': 'Depthwise'}. Best is trial 0 with value: -3.2232916402730614.
[I 2024-12-20 22:26:52,083] Trial 1 finished with value: -3.0659333103635316 and parameters: {'n_estimators': 621, 'learning_rate': 0.3540365808254449, 'depth': 1, 'l2_leaf_reg': 193.98197043239887, 'random_strength': 166.48852816008434, 'bagging_temperature': 42.46782213565523, 'grow_policy': 'Depthwise'}. Best is trial 1 with value: -3.0659333103635316.
[I 2024-12-20 22:27:03,998] Trial 2 finished with value: -3.0977745680782234 and parameters: {'n_estimators': 339, 'learning_rate': 0.26237869105968725, 'depth': 6, 'l2_leaf_reg': 58.245828039608384, 'random_strength': 122.3705789444759, 'bagging_temperature': 27.898772130408368, 'g

In [7]:
print(f"Best score: {study.best_trial.value}")
print(f"Best params: {study.best_trial.params}")

Best score: -2.9699740256649254
Best params: {'n_estimators': 571, 'learning_rate': 0.03510354783722455, 'depth': 7, 'l2_leaf_reg': 18.647375754470975, 'random_strength': 0.12431632785685232, 'bagging_temperature': 79.24277143551846, 'grow_policy': 'SymmetricTree'}


In [8]:
model = CatBoostRegressor(
    n_estimators=571, learning_rate=0.035, depth=7,
    l2_leaf_reg=18.65, random_strength=0.1243, bagging_temperature=79.24, 
    grow_policy='SymmetricTree', verbose=100, random_state=42
)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

0:	learn: 10.4234150	total: 8.64ms	remaining: 4.93s
100:	learn: 3.9504876	total: 876ms	remaining: 4.08s
200:	learn: 3.8749665	total: 1.64s	remaining: 3.02s
300:	learn: 3.8302803	total: 2.45s	remaining: 2.2s
400:	learn: 3.7906169	total: 3.27s	remaining: 1.39s
500:	learn: 3.7557891	total: 4.08s	remaining: 570ms
570:	learn: 3.7336025	total: 4.63s	remaining: 0us
Train MAE: 2.795241939349766
Valid MAE: 3.1203525449164764


In [9]:
model = CatBoostRegressor(
    n_estimators=600, depth=7,
    l2_leaf_reg=20, random_strength=0.125, bagging_temperature=80, 
    grow_policy='SymmetricTree', verbose=100, random_state=42
)
model.fit(X_train, y_train)
print(f'Train MAE: {mean_absolute_error(y_train, model.predict(X_train))}')
print(f'Valid MAE: {mean_absolute_error(y_valid, model.predict(X_valid))}')

0:	learn: 10.4671770	total: 9.03ms	remaining: 5.41s
100:	learn: 3.9947265	total: 847ms	remaining: 4.18s
200:	learn: 3.8859473	total: 1.61s	remaining: 3.19s
300:	learn: 3.8500553	total: 2.41s	remaining: 2.4s
400:	learn: 3.8155564	total: 3.19s	remaining: 1.58s
500:	learn: 3.7837422	total: 3.98s	remaining: 786ms
599:	learn: 3.7567645	total: 4.74s	remaining: 0us
Train MAE: 2.8097723552217433
Valid MAE: 3.1246558203099566
